In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
from AVN import *
from scipy.stats import norm
import scipy.io as scp

In [ ]:
###Define colors code for plots:
from matplotlib import cm

##Get colors : 
cmap_name1 = cm.get_cmap('tab20', 20)
cmap_name2 = cm.get_cmap('tab20b', 20)
cmap_name3 = cm.get_cmap('tab20c', 20)
cmap_name4 = cm.get_cmap('tab10', 10)
cmap_name5 = cm.get_cmap('Set3', 12)

cmap1 = cmap_name1(np.linspace(0, 1, 20))
cmap2 = cmap_name2(np.linspace(0, 1, 20))
cmap3 = cmap_name3(np.linspace(0, 1, 20))
cmap4 = cmap_name4(np.linspace(0, 1, 10))
cmap5 = cmap_name5(np.linspace(0, 1, 12))

cmap = np.vstack([cmap1, cmap2, cmap3, cmap4, cmap5])

print(len(cmap))

In [ ]:
##Add migration swarms:
dfM = pd.read_csv("/home/xianglo/Bureau/DATA_AMATRICE/DBSCAN_David/new_all_run_files_for_etas_23052024_2/new_all_run_files_for_etas/v0_d03_n50_every1_clusters/dfM.csv")

##And without:
dfNM = pd.read_csv("/home/xianglo/Bureau/DATA_AMATRICE/DBSCAN_David/new_all_run_files_for_etas_23052024_2/new_all_run_files_for_etas/v0_d03_n50_every1_clusters/dfNM.csv")

dfM

In [ ]:
path_to_df0 = "/home/xianglo/Bureau/DATA_AMATRICE/DBSCAN_David/new_all_run_files_for_etas_23052024_2/new_all_run_files_for_etas/catalog_all/"

####Raw data:
df0 = pd.read_csv(path_to_df0+'raw_data_modified_version.csv')
###Structure : t, x, y, z, m, x_km, y_km

###Swarms obtained by after ETAS & seismicity rate & checking step:
df_swarms = pd.read_csv('v0_d03_n50_every1_clusters/new_df_final_swarms_of_v0_d03_n50_every1.csv')

##Rename swarms after sorting along t1:
df_swarms = df_swarms.sort_values("t1")
df_swarms.index = np.arange(1, len(df_swarms)+1)

df_swarms

In [ ]:
###Clusters from dbscan:
v0_n50_d0_3 = scp.loadmat("/home/xianglo/Bureau/DATA_AMATRICE/DBSCAN_David/new_all_run_files_for_etas_23052024_2/new_all_run_files_for_etas/dbscan_data/v0_d03km_n50_every1.mat")["clusters"].reshape(-1) ##au moins 50 séismes à < 300 m, v = 0km/d : 1 séisme sur 1 comme vertex

###Add columns:
df0["dbscan"] = v0_n50_d0_3

df0.head()

In [ ]:
###Create a column of swarms number:
df0["swarms_num"] = np.zeros(len(df0)) -1


for i in range(0, len(df_swarms)):
    select_swarms = df0.index[(df0.t >= df_swarms.t1.iloc[i])&(df0.t <= df_swarms.t2.iloc[i])
                       & (df0.x >= df_swarms.x1.iloc[i])&(df0.x <= df_swarms.x2.iloc[i])
                       & (df0.y >= df_swarms.y1.iloc[i])&(df0.y <= df_swarms.y2.iloc[i])
                       & (df0.z >= df_swarms.z1.iloc[i])&(df0.z <= df_swarms.z2.iloc[i])]
    
    df0.swarms_num.iloc[select_swarms] = df_swarms.index[i]   
    
    
df0['swarms_num'] = df0['swarms_num'].astype(int)

print(np.unique(df0["swarms_num"].values)[1:])

###Remove all rows with a magnitude equal to a nan value:
df0 = df0.dropna()

df0.index = np.arange(0, len(df0))

df0

In [ ]:
from AVN import *

###Define AVN MS:
A, V, N, df_AVN = AVN(df0)

df_AVN

In [ ]:
###Rescale on Norcia event:
xN_km = df0.x_km - N.x_km
yN_km = df0.y_km - N.y_km

df0["xN_km"] = xN_km
df0["yN_km"] = yN_km

###Redefine AVN MS:
A, V, N, df_AVN = AVN(df0)

df0_m5 = df0[df0.m >= 5]

df0

In [ ]:
#####Separation swarms North/South :

###Swarms located in North:
IN = df0[(df0.yN_km >= -50)&(df0.yN_km <= 15)
        & (df0.xN_km >= -15)&(df0.xN_km <= 50)]
###Swarms located in South:
IS = df0[(df0.yN_km >= 10)&(df0.yN_km <= 40)
        & (df0.xN_km >= -50)&(df0.xN_km <= -10)]

print("IN:", len(np.unique(IN.swarms_num)[1:]))
print("IS:", len(np.unique(IS.swarms_num)[1:]))

In [ ]:
msz = 500
msz_stars = 1000

plt.figure(figsize=(30,40))
plt.subplot(4,2,1)
##swarms clusters:
list_swarms = np.unique(IN.swarms_num)[1:]
for i in list_swarms:
    if dfM[dfM.I.values == i].empty == False:
        I = df0[df0.swarms_num == i]
        plt.scatter(I.t, I.m, s=np.exp(3*I.m), marker='s', facecolors="none", edgecolors=cmap[int(i)], linewidths=1)
        plt.annotate(str(int(i)), (I.t.max(), I.m.max()),
                color="k", alpha =1, size=20, 
                path_effects=[pe.withStroke(linewidth=3, foreground="k")])
    else:
        I = df0[df0.swarms_num == i]
        plt.scatter(I.t, I.m, s=np.exp(3*I.m), marker='s', facecolors="none", edgecolors=cmap[int(i)], linewidths=1)
        plt.annotate(str(int(i)), (I.t.max(), I.m.max()),
                color="white", alpha =1, size=20, 
                path_effects=[pe.withStroke(linewidth=3, foreground="k")])
    
plt.scatter(df0_m5.t, df0_m5.m, s=msz_stars, marker='*', c='gold', edgecolors='k', linewidths=1)
plt.scatter(A.t, A.m, s=msz_stars, marker='*', c='tab:blue', edgecolors='tab:red', linewidths=1)
plt.scatter(V.t, V.m, s=msz_stars, marker='*', c='tab:orange', edgecolors='tab:red', linewidths=1)
plt.scatter(N.t, N.m, s=msz_stars, marker='*', c='tab:green', edgecolors='tab:red', linewidths=1)
plt.xlim([0,366])
plt.xlabel("Time (days)", fontsize=20)
plt.ylabel("Magnitude $m_L$", fontsize=20)
plt.title("CENTRAL GROUP", fontsize=30)
plt.tick_params(labelsize=20)
plt.grid(True)

plt.subplot(4,2,3)
##swarms clusters:
for i in list_swarms:
    if dfM[dfM.I.values == i].empty == False:
        I = df0[df0.swarms_num == i]
        plt.scatter(I.t.mean(), I.xN_km.mean(), s=msz, marker='s', facecolors=cmap[int(i)], edgecolors='k', linewidths=3)
        plt.annotate(str(int(i)), (I.t.mean(), I.xN_km.mean()),
                color='white', alpha =1, size=20, 
                path_effects=[pe.withStroke(linewidth=3, foreground="k")])
    else:
        I = df0[df0.swarms_num == i]
        plt.scatter(I.t.mean(), I.xN_km.mean(), s=msz, marker='s', facecolors=cmap[int(i)], edgecolors=cmap[int(i)], linewidths=1)
        plt.annotate(str(int(i)), (I.t.mean(), I.xN_km.mean()),
                color='white', alpha =1, size=20, 
                path_effects=[pe.withStroke(linewidth=3, foreground="k")])
    
plt.scatter(df0_m5.t, df0_m5.xN_km, s=msz_stars, marker='*', c='gold', edgecolors='k', linewidths=1)
plt.scatter(A.t, A.xN_km, s=msz_stars, marker='*', c='tab:blue', edgecolors='tab:red', linewidths=1)
plt.scatter(V.t, V.xN_km, s=msz_stars, marker='*', c='tab:orange', edgecolors='tab:red', linewidths=1)
plt.scatter(N.t, N.xN_km, s=msz_stars, marker='*', c='tab:green', edgecolors='tab:red', linewidths=1)
plt.xlim([0,366])
plt.ylabel("N-S (km)", fontsize=15)
plt.xlabel("Time (d)", fontsize=20)
plt.tick_params(labelsize=20)
plt.grid(True)

plt.subplot(4,2,5)
##swarms clusters:
for i in list_swarms:
    if dfM[dfM.I.values == i].empty == False:
        I = df0[df0.swarms_num == i]
        plt.scatter(I.t.mean(), I.yN_km.mean(), s=msz, marker='s', facecolors=cmap[int(i)], edgecolors='k', linewidths=3)
        plt.annotate(str(int(i)), (I.t.mean(), I.yN_km.mean()),
                color="white", alpha =1, size=20, 
                path_effects=[pe.withStroke(linewidth=3, foreground="k")])
    else:
        I = df0[df0.swarms_num == i]
        plt.scatter(I.t.mean(), I.yN_km.mean(), s=msz, marker='s', facecolors=cmap[int(i)], edgecolors=cmap[int(i)], linewidths=1)
        plt.annotate(str(int(i)), (I.t.mean(), I.yN_km.mean()),
                color="white", alpha =1, size=20, 
                path_effects=[pe.withStroke(linewidth=3, foreground="k")])

plt.scatter(df0_m5.t, df0_m5.yN_km, s=msz_stars, marker='*', c='gold', edgecolors='k', linewidths=1)
plt.scatter(A.t, A.yN_km, s=msz_stars, marker='*', c='tab:blue', edgecolors='tab:red', linewidths=1)
plt.scatter(V.t, V.yN_km, s=msz_stars, marker='*', c='tab:orange', edgecolors='tab:red', linewidths=1)
plt.scatter(N.t, N.yN_km, s=msz_stars, marker='*', c='tab:green', edgecolors='tab:red', linewidths=1)
plt.xlim([0,366])
plt.xlabel("Time (days)", fontsize=20)
plt.ylabel("W-E (km)", fontsize=15)
plt.tick_params(labelsize=20)
plt.grid(True)

plt.subplot(4,2,7)
##swarms clusters:
for i in list_swarms:
    if dfM[dfM.I.values == i].empty == False:
        I = df0[df0.swarms_num == i]
        plt.scatter(I.t.mean(), I.z.mean(), s=msz, marker='s', facecolors=cmap[int(i)], edgecolors='k', linewidths=3)
        plt.annotate(str(int(i)), (I.t.mean(), I.z.mean()),
                color="white", alpha =1, size=20, 
                path_effects=[pe.withStroke(linewidth=3, foreground="k")])
    else:
        I = df0[df0.swarms_num == i]
        plt.scatter(I.t.mean(), I.z.mean(), s=msz, marker='s', facecolors=cmap[int(i)], edgecolors=cmap[int(i)], linewidths=1)
        plt.annotate(str(int(i)), (I.t.mean(), I.z.mean()),
                color="white", alpha =1, size=20, 
                path_effects=[pe.withStroke(linewidth=3, foreground="k")])
    
plt.scatter(df0_m5.t, df0_m5.z, s=msz_stars, marker='*', c='gold', edgecolors='k', linewidths=1)
plt.scatter(A.t, A.z, s=msz_stars, marker='*', c='tab:blue', edgecolors='tab:red', linewidths=1)
plt.scatter(V.t, V.z, s=msz_stars, marker='*', c='tab:orange', edgecolors='tab:red', linewidths=1)
plt.scatter(N.t, N.z, s=msz_stars, marker='*', c='tab:green', edgecolors='tab:red', linewidths=1)

##Put the origin point to the top left corner:
ax=plt.gca()                            # get the axis
ax.set_ylim(ax.get_ylim()[::-1])        # invert the axis
ax.xaxis.tick_top()                     # and move the X-Axis      
ax.yaxis.tick_left()                    # remove right y-Ticks

plt.xlim([0,366])
plt.ylim([12,0])
plt.ylabel("Depth (km)", fontsize=20)
plt.tick_params(labelsize=20)
plt.grid(True)

###########################################################################################################
###########################################################################################################
###########################################################################################################

plt.subplot(4,2,2)
##swarms clusters:
list_swarms = np.unique(IS.swarms_num)[1:]
for i in list_swarms:
    if dfM[dfM.I.values == i].empty == False:
        I = df0[df0.swarms_num == i]
        plt.scatter(I.t, I.m, s=np.exp(3*I.m), marker='s', facecolors="none", edgecolors=cmap[int(i)], linewidths=1)
        plt.annotate(str(int(i)), (I.t.max(), I.m.max()),
                color="k", alpha =1, size=20, 
                path_effects=[pe.withStroke(linewidth=3, foreground="k")])
    else:
        I = df0[df0.swarms_num == i]
        plt.scatter(I.t, I.m, s=np.exp(3*I.m), marker='s', facecolors="none", edgecolors=cmap[int(i)], linewidths=1)
        plt.annotate(str(int(i)), (I.t.max(), I.m.max()),
                color="white", alpha =1, size=20, 
                path_effects=[pe.withStroke(linewidth=3, foreground="k")])
    
plt.scatter(df0_m5.t, df0_m5.m, s=msz_stars, marker='*', c='gold', edgecolors='k', linewidths=1)
plt.scatter(A.t, A.m, s=msz_stars, marker='*', c='tab:blue', edgecolors='tab:red', linewidths=1)
plt.scatter(V.t, V.m, s=msz_stars, marker='*', c='tab:orange', edgecolors='tab:red', linewidths=1)
plt.scatter(N.t, N.m, s=msz_stars, marker='*', c='tab:green', edgecolors='tab:red', linewidths=1)
plt.xlim([0,366])
plt.xlabel("Time (days)", fontsize=20)
plt.ylabel("Magnitude $m_L$", fontsize=20)
plt.title("SOUTHERN GROUP", fontsize=30)
plt.tick_params(labelsize=20)
plt.grid(True)

plt.subplot(4,2,4)
##swarms clusters:
for i in list_swarms:
    if dfM[dfM.I.values == i].empty == False:
        I = df0[df0.swarms_num == i]
        plt.scatter(I.t.mean(), I.xN_km.mean(), s=msz, marker='s', facecolors=cmap[int(i)], edgecolors='k', linewidths=3)
        plt.annotate(str(int(i)), (I.t.mean(), I.xN_km.mean()),
                color='white', alpha =1, size=20, 
                path_effects=[pe.withStroke(linewidth=3, foreground="k")])
    else:
        I = df0[df0.swarms_num == i]
        plt.scatter(I.t.mean(), I.xN_km.mean(), s=msz, marker='s', facecolors=cmap[int(i)], edgecolors=cmap[int(i)], linewidths=1)
        plt.annotate(str(int(i)), (I.t.mean(), I.xN_km.mean()),
                color='white', alpha =1, size=20, 
                path_effects=[pe.withStroke(linewidth=3, foreground="k")])

plt.scatter(df0_m5.t, df0_m5.xN_km, s=msz_stars, marker='*', c='gold', edgecolors='k', linewidths=1)
plt.scatter(A.t, A.xN_km, s=msz_stars, marker='*', c='tab:blue', edgecolors='tab:red', linewidths=1)
plt.scatter(V.t, V.xN_km, s=msz_stars, marker='*', c='tab:orange', edgecolors='tab:red', linewidths=1)
plt.scatter(N.t, N.xN_km, s=msz_stars, marker='*', c='tab:green', edgecolors='tab:red', linewidths=1)
plt.xlim([0,366])
plt.xlabel("Time (d)", fontsize=20)
plt.ylabel("N-S (km)", fontsize=15)
plt.tick_params(labelsize=20)
plt.grid(True)

plt.subplot(4,2,6)
##swarms clusters:
for i in list_swarms:
    if dfM[dfM.I.values == i].empty == False:
        I = df0[df0.swarms_num == i]
        plt.scatter(I.t.mean(), I.yN_km.mean(), s=msz, marker='s', facecolors=cmap[int(i)], edgecolors='k', linewidths=3)
        plt.annotate(str(int(i)), (I.t.mean(), I.yN_km.mean()),
                color="white", alpha =1, size=20, 
                path_effects=[pe.withStroke(linewidth=3, foreground="k")])
    else:
        I = df0[df0.swarms_num == i]
        plt.scatter(I.t.mean(), I.yN_km.mean(), s=msz, marker='s', facecolors=cmap[int(i)], edgecolors=cmap[int(i)], linewidths=1)
        plt.annotate(str(int(i)), (I.t.mean(), I.yN_km.mean()),
                color="white", alpha =1, size=20, 
                path_effects=[pe.withStroke(linewidth=3, foreground="k")])

plt.scatter(df0_m5.t, df0_m5.yN_km, s=msz_stars, marker='*', c='gold', edgecolors='k', linewidths=1)
plt.scatter(A.t, A.yN_km, s=msz_stars, marker='*', c='tab:blue', edgecolors='tab:red', linewidths=1)
plt.scatter(V.t, V.yN_km, s=msz_stars, marker='*', c='tab:orange', edgecolors='tab:red', linewidths=1)
plt.scatter(N.t, N.yN_km, s=msz_stars, marker='*', c='tab:green', edgecolors='tab:red', linewidths=1)
plt.xlim([0,366])
plt.xlabel("Time (d)", fontsize=20)
plt.ylabel("W-E (km)", fontsize=15)
plt.tick_params(labelsize=20)
plt.grid(True)

plt.subplot(4,2,8)
##swarms clusters:
for i in list_swarms:
    if dfM[dfM.I.values == i].empty == False:
        I = df0[df0.swarms_num == i]
        plt.scatter(I.t.mean(), I.z.mean(), s=msz, marker='s', facecolors=cmap[int(i)], edgecolors='k', linewidths=3)
        plt.annotate(str(int(i)), (I.t.mean(), I.z.mean()),
                color="white", alpha =1, size=20, 
                path_effects=[pe.withStroke(linewidth=3, foreground="k")])
    else:
        I = df0[df0.swarms_num == i]
        plt.scatter(I.t.mean(), I.z.mean(), s=msz, marker='s', facecolors=cmap[int(i)], edgecolors=cmap[int(i)], linewidths=1)
        plt.annotate(str(int(i)), (I.t.mean(), I.z.mean()),
                color="white", alpha =1, size=20, 
                path_effects=[pe.withStroke(linewidth=3, foreground="k")])
    
plt.scatter(df0_m5.t, df0_m5.z, s=msz_stars, marker='*', c='gold', edgecolors='k', linewidths=1)
plt.scatter(A.t, A.z, s=msz_stars, marker='*', c='tab:blue', edgecolors='tab:red', linewidths=1)
plt.scatter(V.t, V.z, s=msz_stars, marker='*', c='tab:orange', edgecolors='tab:red', linewidths=1)
plt.scatter(N.t, N.z, s=msz_stars, marker='*', c='tab:green', edgecolors='tab:red', linewidths=1)

##Put the origin point to the top left corner:
ax=plt.gca()                            # get the axis
ax.set_ylim(ax.get_ylim()[::-1])        # invert the axis
ax.xaxis.tick_top()                     # and move the X-Axis      
ax.yaxis.tick_left()                    # remove right y-Ticks

plt.xlim([0,366])
plt.ylim([12,0])
plt.ylabel("Depth (km)", fontsize=20)
plt.tick_params(labelsize=20)
plt.grid(True)


plt.tight_layout()
plt.savefig("v0_d03_n50_every1_clusters/m_xyzvst_for_IN_IS_swarms.png")
plt.show()